In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [12]:
np.random.seed(1232)
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\dep')
data = pd.read_csv("Rand_train.csv", sep=',')
data_val = pd.read_csv("Rand_valid.csv", sep=',')

In [13]:
data.cost = (np.exp(data.cost) - 0.5)*((data.cost != 0) * 1)
data_val.cost = (np.exp(data_val.cost) - 0.5)*((data_val.cost != 0) * 1)
data = shuffle(data).reset_index(drop=True)
data = data.drop(['fmde', 'lxghindx', 'lpi'], axis=1)
data_val = data_val.drop(['fmde', 'lxghindx', 'lpi'], axis=1)

In [14]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

ranking = np.load('ranking.npy')

my_index = X.columns.values[ranking]
X2 = X[my_index]
X2_val = X_val[my_index]

In [15]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [16]:
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\prog_calc')
from fit6_nn import fit, comb_error

In [17]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(17, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
        
        self.drop_layer = nn.Dropout(p=0.02)
                
    def forward(self, x):        
               
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2)) 
        x2 = self.fc23(x2)
        
        x1 = torch.cat((x2, x), 1)
        x1 = self.fc11(x1)
        x1 = self.drop_layer(x1)
        x1 = torch.tanh(x1)
        
        x1 = torch.sigmoid(self.fc12(x1))        
        x2 = torch.exp(x2)        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [18]:
n_epochs = 5000
lr = 0.001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 10
clipping = 0.20
sig2 = 950000
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining11.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [ ]:
net = Net1()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

#PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining11.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, 
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

Epoch 1: Training Loss: 1.1882403492927551, Validation Loss: 1.3193899393081665
Epoch 2: Training Loss: 1.1859925786654155, Validation Loss: 1.314814805984497
Epoch 3: Training Loss: 1.1800767580668132, Validation Loss: 1.308499813079834
Epoch 4: Training Loss: 1.1748620768388112, Validation Loss: 1.3009841442108154
Epoch 5: Training Loss: 1.1667421062787373, Validation Loss: 1.2926658391952515
Epoch 6: Training Loss: 1.158872773249944, Validation Loss: 1.2838354110717773
Epoch 7: Training Loss: 1.150535782178243, Validation Loss: 1.274705410003662
Epoch 8: Training Loss: 1.1426888207594554, Validation Loss: 1.2654292583465576
Epoch 9: Training Loss: 1.1327085395654042, Validation Loss: 1.2561172246932983
Epoch 10: Training Loss: 1.123949219783147, Validation Loss: 1.2468488216400146
Epoch 11: Training Loss: 1.1161368389924367, Validation Loss: 1.2376811504364014
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 1.1073394119739532, Validation Loss: 1.2286560535430908
Epoch 13: Tr

Epoch 100: Training Loss: 0.8938683470090231, Validation Loss: 1.001023292541504
Epoch 101: Training Loss: 0.8927789727846781, Validation Loss: 1.0003082752227783
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.8926422894001007, Validation Loss: 0.9995934367179871
Epoch 103: Training Loss: 0.8914803365866343, Validation Loss: 0.9989022612571716
Epoch 104: Training Loss: 0.8910276790459951, Validation Loss: 0.9982143044471741
Epoch 105: Training Loss: 0.8907194435596466, Validation Loss: 0.9975168108940125
Epoch 106: Training Loss: 0.8895632028579712, Validation Loss: 0.9968582987785339
Epoch 107: Training Loss: 0.8885105152924856, Validation Loss: 0.9962014555931091
Epoch 108: Training Loss: 0.8881704310576121, Validation Loss: 0.9955347776412964
Epoch 109: Training Loss: 0.8879550993442535, Validation Loss: 0.994868814945221
Epoch 110: Training Loss: 0.8870366811752319, Validation Loss: 0.9942195415496826
Epoch 111: Training Loss: 0.8862060308456421, Validation Loss: 0.9935

Epoch 197: Training Loss: 0.861367533604304, Validation Loss: 0.9685923457145691
Epoch 198: Training Loss: 0.8605040709177653, Validation Loss: 0.9685207009315491
Epoch 199: Training Loss: 0.8592557609081268, Validation Loss: 0.9684788584709167
Epoch 200: Training Loss: 0.8605274856090546, Validation Loss: 0.9684361219406128
Epoch 201: Training Loss: 0.8601001997788748, Validation Loss: 0.9683918356895447
New Checkpoint Saved into PATH
Epoch 202: Training Loss: 0.8603475689888, Validation Loss: 0.9683310985565186
Epoch 203: Training Loss: 0.8594149450461069, Validation Loss: 0.9682675004005432
Epoch 204: Training Loss: 0.8604737420876821, Validation Loss: 0.9682241678237915
Epoch 205: Training Loss: 0.8595020274321238, Validation Loss: 0.9681810736656189
Epoch 206: Training Loss: 0.8592628141244253, Validation Loss: 0.9681159257888794
Epoch 207: Training Loss: 0.8604117035865784, Validation Loss: 0.9680666327476501
Epoch 208: Training Loss: 0.8584170838197073, Validation Loss: 0.967995

Epoch 295: Training Loss: 0.8568289577960968, Validation Loss: 0.9655972123146057
Epoch 296: Training Loss: 0.8566913902759552, Validation Loss: 0.9655711054801941
Epoch 297: Training Loss: 0.8566289941469828, Validation Loss: 0.965549111366272
Epoch 298: Training Loss: 0.856918454170227, Validation Loss: 0.9654809832572937
Epoch 299: Training Loss: 0.8575253585974375, Validation Loss: 0.965474009513855
Epoch 300: Training Loss: 0.8563489615917206, Validation Loss: 0.9654359817504883
Epoch 301: Training Loss: 0.8566527962684631, Validation Loss: 0.9654428362846375
New Checkpoint Saved into PATH
Epoch 302: Training Loss: 0.8560962677001953, Validation Loss: 0.9653953909873962
Epoch 303: Training Loss: 0.855698158343633, Validation Loss: 0.9653714299201965
Epoch 304: Training Loss: 0.8565151691436768, Validation Loss: 0.9653577208518982
Epoch 305: Training Loss: 0.8563457926114401, Validation Loss: 0.9653654098510742
Epoch 306: Training Loss: 0.8555017610390981, Validation Loss: 0.965325

Epoch 392: Training Loss: 0.8551958302656809, Validation Loss: 0.9644311666488647
Epoch 393: Training Loss: 0.8548895517985026, Validation Loss: 0.9644634127616882
Epoch 394: Training Loss: 0.8550186157226562, Validation Loss: 0.9644730687141418
Epoch 395: Training Loss: 0.8547046581904093, Validation Loss: 0.9644308686256409
Epoch 396: Training Loss: 0.854353129863739, Validation Loss: 0.9644115567207336
Epoch 397: Training Loss: 0.8552317122618357, Validation Loss: 0.964364767074585
Epoch 398: Training Loss: 0.8550074795881907, Validation Loss: 0.9643136858940125
Epoch 399: Training Loss: 0.8541362384955088, Validation Loss: 0.964336633682251
Epoch 400: Training Loss: 0.8537845611572266, Validation Loss: 0.9643466472625732
Epoch 401: Training Loss: 0.8548168838024139, Validation Loss: 0.9643262028694153
New Checkpoint Saved into PATH
Epoch 402: Training Loss: 0.8545069595177969, Validation Loss: 0.9643135666847229
Epoch 403: Training Loss: 0.8545663555463155, Validation Loss: 0.96435

Epoch 491: Training Loss: 0.8535780310630798, Validation Loss: 0.9637374877929688
New Checkpoint Saved into PATH
Epoch 492: Training Loss: 0.8529616097609202, Validation Loss: 0.963762640953064
Epoch 493: Training Loss: 0.8536954522132874, Validation Loss: 0.9637771844863892
Epoch 494: Training Loss: 0.8538841903209686, Validation Loss: 0.9637750387191772
Epoch 495: Training Loss: 0.8535060187180837, Validation Loss: 0.9637487530708313
Epoch 496: Training Loss: 0.8540997505187988, Validation Loss: 0.9637142419815063
Epoch 497: Training Loss: 0.8541678289572397, Validation Loss: 0.9637264609336853
Epoch 498: Training Loss: 0.8535669247309366, Validation Loss: 0.9637176394462585
Epoch 499: Training Loss: 0.8529739379882812, Validation Loss: 0.9637502431869507
Epoch 500: Training Loss: 0.8534099459648132, Validation Loss: 0.9637125730514526
Epoch 501: Training Loss: 0.8529677291711172, Validation Loss: 0.9637727737426758
New Checkpoint Saved into PATH
Epoch 502: Training Loss: 0.854185342

Epoch 588: Training Loss: 0.8521605134010315, Validation Loss: 0.9633799195289612
Epoch 589: Training Loss: 0.8526490132013956, Validation Loss: 0.96339350938797
Epoch 590: Training Loss: 0.8524702588717142, Validation Loss: 0.9633594751358032
Epoch 591: Training Loss: 0.8523931403954824, Validation Loss: 0.9633451700210571
Epoch 592: Training Loss: 0.8524965445200602, Validation Loss: 0.963417112827301
Epoch 593: Training Loss: 0.852724293867747, Validation Loss: 0.9634202718734741
Epoch 594: Training Loss: 0.8527407844861349, Validation Loss: 0.9633883237838745
Epoch 595: Training Loss: 0.8532370229562124, Validation Loss: 0.9633391499519348
Epoch 596: Training Loss: 0.8528905610243479, Validation Loss: 0.963356077671051
Epoch 597: Training Loss: 0.8526435097058614, Validation Loss: 0.9633124470710754
Epoch 598: Training Loss: 0.8526797990004221, Validation Loss: 0.9632923603057861
Epoch 599: Training Loss: 0.853127638498942, Validation Loss: 0.9632644057273865
Epoch 600: Training Lo

Epoch 688: Training Loss: 0.8516223430633545, Validation Loss: 0.9630088806152344
Epoch 689: Training Loss: 0.8515638609727224, Validation Loss: 0.9630422592163086
Epoch 690: Training Loss: 0.8523008128007253, Validation Loss: 0.9630191922187805
Epoch 691: Training Loss: 0.8523169060548147, Validation Loss: 0.9630638957023621
New Checkpoint Saved into PATH
Epoch 692: Training Loss: 0.8513896564642588, Validation Loss: 0.9630887508392334
Epoch 693: Training Loss: 0.8525331616401672, Validation Loss: 0.9629929065704346
Epoch 694: Training Loss: 0.8517043789227804, Validation Loss: 0.9629378318786621
Epoch 695: Training Loss: 0.8520965576171875, Validation Loss: 0.9629272222518921
Epoch 696: Training Loss: 0.851597805817922, Validation Loss: 0.9629195928573608
Epoch 697: Training Loss: 0.851955384016037, Validation Loss: 0.962857186794281
Epoch 698: Training Loss: 0.8513354659080505, Validation Loss: 0.9628937244415283

In [ ]:
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining11.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [ ]:
with torch.no_grad():
    val_inputs = torch.FloatTensor(X2_val.values)
    val_labels = torch.FloatTensor(Y2_val)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
    val_outputs = net.forward(val_inputs)
    val_loss = m_error(val_outputs, val_labels) 
    print(val_loss.item())

In [ ]:
val_outputs

In [ ]:
val_labels

In [ ]:
val_outputs = val_outputs.numpy()
val_labels = val_labels.numpy()
y_scores = val_outputs[:,0]
y_real = val_labels[:,0]

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    plt.figure(figsize=(8,8))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.legend(loc='best')

In [ ]:
fpr, tpr, auc_thresholds = roc_curve(y_real, y_scores)
print(auc(fpr, tpr)) # AUC of ROC
plot_roc_curve(fpr, tpr, 'recall_optimized')

In [ ]:
y = val_labels[:,1][val_labels[:,1]>0]
yhat = val_outputs[:,1][val_labels[:,1]>0]

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
y = np.array(Y_val)  #so we can tune the cut point
p = val_outputs[:,0]
f2 = val_outputs[:,1]
yhat = (1-p)*f2

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
print(np.cov(y, p))

In [ ]:
print(np.cov(f2, p))